In [1]:
import sys
!{sys.executable} -m pip install --user -i http://pypi.repo.test.hhu.de/simple/ --trusted-host pypi.repo.test.hhu.de typing-extensions==3.7.4 --upgrade xgboost
!{sys.executable} -m pip install --user -i http://pypi.repo.test.hhu.de/simple/ --trusted-host pypi.repo.test.hhu.de typing-extensions==3.7.4 --upgrade biopython

!{sys.executable} -m pip install --user -i http://pypi.repo.test.hhu.de/simple/ --trusted-host pypi.repo.test.hhu.de typing-extensions==3.7.4 --upgrade hyperopt
    


Looking in indexes: http://pypi.repo.test.hhu.de/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 5.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.0.3
    Uninstalling xgboost-2.0.3:
      Successfully uninstalled xgboost-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anyio 4.2.0 requires typing-extensions>=4.1; python_version < "3.11", but you have typing-extensions 3.7.4 which is incompatible.
async-lru 2.0.4 requires typing-extensions>=4.0.0; python_version < "3.11", but you have typing-extensions 3.7.4 which is incompatible.
pydantic 2.6.3 requires typing-extensions>=4.6.1, 

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK, rand
import xgboost as xgb
from xgboost import XGBClassifier
import numpy as np
import os
from sklearn.metrics import r2_score

# path = "/home/not81yan/km_predict_proj/Model/Dataset"
path = os.getcwd()+"/../rough/final_data_32000.npy"
arr = np.load(path)


In [45]:
arr.shape

(3200, 2563)

In [47]:
# test_arr = arr[:1000,:]
X = arr[:, 3:]
y = arr[:, 2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
D_train=xgb.DMatrix(data=X_test, label=y_test)
D_test=xgb.DMatrix(data=X_train, label=y_train)

#now do k-fold cross validation split of X_train, y_train   
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [48]:
X.shape

(3200, 2560)

In [49]:
splits_test = []
splits_train = []

In [50]:
for train_index, test_index in kf.split(X_train):
    splits_test.append(test_index)
    splits_train.append(train_index)

In [56]:
def cross_validation_mse_gradient_boosting(params):
    num_round = params['num_rounds']
    params = {
        'max_depth': int(np.round(params['max_depth'])),
        'learning_rate': params['learning_rate'],
        'min_child_weight': params['min_child_weight'],
        'reg_alpha': params['reg_alpha'],  # L1 regularization
        'reg_lambda': params['reg_lambda'],  # L2 regularization
        'tree_method':   # Use CPU for training
            #select cpu for training
            'hist',
        
        # 'predictor': 'gpu_predictor',  # Use GPU for predictions
        'sampling_method': 'uniform',
        'eval_metric': 'rmse'
    }
    MSE = []
    R2 = []    
    for i in range(5):
        train_index, test_index  = splits_train[i], splits_test[i]
        dtrain = xgb.DMatrix(X_train[train_index], label = y_train[train_index])
        dvalid = xgb.DMatrix(X_train[test_index])
        bst = xgb.train(params, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(y_train[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(y_train[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


In [57]:

space = {
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    # 'n_estimators': hp.quniform('n_estimators', 100, 500, 1),
    # 'gamma': hp.uniform('gamma', 0, 0.5),
    # 'min_child_weight': hp.quniform('min_child_weight', 1, 10, 1),
    # 'subsample': hp.uniform('subsample', 0.5, 1),
    # 'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    # 'reg_alpha': hp.uniform('reg_alpha', 0, 1),  # L1 regularization
    # 'reg_lambda': hp.uniform('reg_lambda', 0, 1)  # L2 regularization
    # "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    # "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)
}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space,
            algo=rand.suggest, max_evals = 250, trials=trials)

  0%|          | 0/250 [00:00<?, ?trial/s, best loss=?]

In [ ]:
# Print the best parameters
print("Best parameters found by Hyperopt:")
print(best)
num_round = best['num_rounds']
del best['num_rounds']
params={'max_depth': int(np.round(best['max_depth'])),
        'learning_rate': best['learning_rate'],
        'min_child_weight': best['min_child_weight'],
        'reg_alpha': best['reg_alpha'],  # L1 regularization
        'reg_lambda': best['reg_lambda'],  # L2 regularization
        'tree_method': 'gpu_hist',  # Use GPU
        'predictor': 'gpu_predictor',  # Use GPU for predictions
        'sampling_method': 'gradient_based',
        'eval_metric': 'rmse'}
best_model = xgb.train(params, xgb.DMatrix(X_train,label=y_train), int(num_round), verbose_eval=False)

best_model.fit(X_train, y_train)

# Evaluate the final model on the test set
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy: {:.2f}".format(test_accuracy))
